# Terminal de Experimentos


In [2]:
class experimento_con_ruido:
    
    #Dataframe de entrada
    def __init__(self, df, y_column, metodo,df_name):
        #Librerias que utilizaremos 
        import warnings
        from sklearn.exceptions import ConvergenceWarning
        from pandas import Series
        import numpy as np
        warnings.filterwarnings("ignore", category = ConvergenceWarning, module = "sklearn")
        
        """
        X : atributos
        y : etiquetas
        """
        
        X_columns = [c for c in df.columns if c!=y_column] 
        self.y = df[y_column].to_numpy() #Etiquetas
        self.X = df[X_columns].to_numpy() #Atributos
        self.metodo = metodo #Metodo de aprendizaje automático de la libreria sklearn
        # self.path = path #Lugar donde se guardaran los documentos en el ordenador
        self.clases = np.unique(self.y) #clases de los atributos
        self.df_name = df_name #nombre del dataset que utilizaremos durante el experimento

    def meter_ruido(self,pr):
        
        import numpy as np
        import copy
        import random
        import os
        
        """
        Función que mete ruido en unas etiquetas dadas
        y : etiquetas donde queremos meter el ruido
        pr : porcentaje de ruido que metemos en las etiquetas
        """
        folder_id = "15Rt7UEmWmCdeRH_v9JmBmfR7WPmzTyth"
        files_dict = get_drive_files_dict(folder_id)
        file_name = f"ruido_en_{self.df_name}_{pr}.csv"
        file_id = files_dict.get(file_name)
        url = f"https://drive.google.com/uc?id={file_id}"  # Replace with actual file ID
        noise_df = pd.read_csv(url)
        self.y_mal = noise_df["Etiquetas con ruido"].to_numpy()
        self.ruido = noise_df["Valores donde hay ruido"].to_numpy()


    #Hay que arreglar lo del self.classes para que no haya redundancia entre las funciones de evaluar y meter_ruido
    
    def evaluar(self, y_test, y_pred, clases):

        import matplotlib.pyplot as plt
        from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,classification_report,cohen_kappa_score
        
        """
        Evalua un método de clasificación
        y_test: valores esperados
        y_pred: valores predichos por el model
        modelo: el modelo dm ML
        clases: lista posibles clases
        
        Devuelve: kappa de Cohen y matriz de confusion 
        Muestra por pantalla el class. report
        """
        k =  cohen_kappa_score(y_test,y_pred)
        print("kappa ",k)
        cm = confusion_matrix(y_test, y_pred, labels=clases)
        print(cm)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                    display_labels=clases)
        disp.plot()
    
        plt.show()
    
        print(classification_report(y_test, y_pred))
        return k,cm


    def aplica_y_evalúa_método(self,pr,test):
        
        from sklearn.model_selection import train_test_split
        from sklearn.neural_network import MLPClassifier

        """
        Devuelve las gráficas PvsP, PvsD, DvsD, DvsP
        pr : proporción ruido en las etiquetas
        test : proporción de datos que utilizaremos para el test
        """
        
        # 1 preparar train y test
        X_train, X_test, y_train, y_test = train_test_split(self.X, self.y, test_size= test)
    
        y_test_mal = self.meter_ruido(y_test, pr)
        y_train_mal = self.meter_ruido(y_train, pr)
    
        
        # 2 método y entrenamiento
        modelo = self.metodo.fit(X_train,y_train)
        
        # 3 evaluar
        y_pred = modelo.predict(X_test)  
    
        #PvsP
        print("PvsP")
        self.evaluar(y_test,y_pred,modelo.classes_)
    
        #PvsD
        print("PvsD")
        self.evaluar(y_test_mal,y_pred,modelo.classes_)
    
        # 2 método y entrenamiento
        modelo2 = self.metodo.fit(X_train,y_train_mal)
    
        # 3 evaluar
        y_pred_mal = modelo2.predict(X_test)
        
        #DvsD
        print("DvsD")
        self.evaluar(y_test_mal,y_pred_mal,modelo2.classes_)
        
        #DvsP
        print("DvsP")
        self.evaluar(y_test,y_pred_mal,modelo2.classes_)
        return 


    """
    Estadísticas para ver si el algoritmo que hemos obtenido funciona 
    """


    def estadisticas_noise_algorithm(self, num_veces, con, metodo, logger):

        """
        Función que devuelve una lista con sublistas de los kappas obtenidos para cada tipo de entrenamiento y test (PvsP, DvsP, PvsD y DvsD)
        num_veces : numero de veces que se aplica el algoritmo 
        con : numero entre [0,1] que mide del num_veces cuantos los tienen que detectar como error para que se reporte como ruido
        metodo : algoritmo de aprendizaje automático de clasificación utilizado
        """

        from tqdm import tqdm
        import csv
        from sklearn.model_selection import StratifiedKFold,train_test_split
        from sklearn.neural_network import MLPClassifier
        from sklearn.metrics import cohen_kappa_score
        import numpy as np
        
        kappa_list_pd = [] # p versus d
        kappa_list_dd = [] # d versus d
        kappa_list_pp = [] # p versus p
        kappa_list_dp = [] # d versus p
        n_splits = 5
        
        for i in range(num_veces):
            particiones = StratifiedKFold(n_splits=n_splits, shuffle=True)     
            
            for j, (train_index, test_index) in enumerate(particiones.split(self.X, self.y)):
                logger.info(f"Partición {i} del dataset para obtener kappas")
                logger.info(f"Particion test {j}")
                
                #Conjunto de datos de entrenamiento sin filtrar
                X_train_raw = self.X[train_index,]
                y_train_raw = self.y_mal[train_index]
                
                #Aplicamos el método para detectar el ruido
                errores = noise_detection_algorithm(X_train_raw, y_train_raw, num_veces, con, metodo, logger) 
                buenos = np.where(errores == False)[0]

                #Conjunto de entrenamiento con los datos filtrados
                X_train = X_train_raw[buenos,]
                y_train = y_train_raw[buenos]

                #Datos del test
                X_test = self.X[test_index,]
                y_test = self.y[test_index]
                y_test_mal = self.y_mal[test_index] #no nos asegura que haya un porcentaje de error constante, puede variar mucho, importa en algo?
                
                # generar modelo y predecir con los datos filtrados
                modelo = self.metodo.fit(X_train,y_train)         
                y_pred = modelo.predict(X_test) #  predecir solo el test

                #Cohen kappa de PvsP y PvsD
                kappa_list_pp.append(cohen_kappa_score(y_test,y_pred))
                kappa_list_pd.append(cohen_kappa_score(y_test_mal,y_pred))

                # generar modelo y predecir con los datos sin filtrar
                modelo2 = self.metodo.fit(X_train_raw,y_train_raw)
                y_pred = modelo2.predict(X_test)

                #Cohen kappa de DvsP y DvsD
                kappa_list_dp.append(cohen_kappa_score(y_test,y_pred))
                kappa_list_dd.append(cohen_kappa_score(y_test_mal,y_pred))

        
        header = ["Cohen Kappa"]
        data_pd = kappa_list_pd
        data_dd = kappa_list_dd
        data_pp = kappa_list_pp
        data_dp = kappa_list_dp
          
        # Write data to CSV
        # with open(self.path, mode='a', newline='') as file:
        #     writer = csv.writer(file)
            
        #     # Write in the csv
        #     writer.writerow(header)

        #     writer.writerow(["Cohen kappa PvsP"])
        #     writer.writerows([data_pp])

        #     writer.writerow(["Cohen kappa PvsD"])
        #     writer.writerows([data_pd])

        #     writer.writerow(["Cohen kappa DvsP"])
        #     writer.writerows([data_dp])
            
        #     writer.writerow(["Cohen kappa DvsD"])
        #     writer.writerows([data_dd])                        

        return [data_pp,data_pd,data_dp,data_dd]

    def est(self,logger):

        """
        Funcion que calcula los kappa para el dataset completo sin y con ruido
        """
        
        from sklearn.metrics import confusion_matrix
        import csv
        kappa1 = []
        kappa2 = []
        
        #Imprime por pantalla los resultados obtenidos con la matriz de confusion
        for i in range(10):
            logger.info(f"Calculando kappa dataset completo sin ruido, {i}")
            kappa1.append(kappa_dataset_completo(self.X, self.y, 10, self.metodo))
            logger.info(f"Calculando kappa dataset completo con ruido, {i}")
            kappa2.append(kappa_dataset_completo(self.X, self.y_mal, 10, self.metodo))
        
        # Asocia la matriz de confusion a la variable data
        # header1 = ["Kappa dataset completo sin ruido"]
        # header2 = ["Kappa dataset completo con ruido"] 
        
        # # Comentario : Falta poner para saber cual es cual en confusion matrix
        # # Añdimos la excel la matriz de confusión
        # with open(self.path, mode='a', newline='') as file:
        #     writer = csv.writer(file)
            
        #     # Write header (optional)
        #     writer.writerow(header1)
            
        #     # Write the data
        #     writer.writerow([kappa1])

        #     # Write header (optional)
        #     writer.writerow(header2)
            
        #     # Write the data
        #     writer.writerow([kappa2])

        return kappa1,kappa2

## Generates a dictionary with the file names and the files id of the google drive folder

In [3]:
def get_drive_files_dict(folder_id):
    import requests
    from bs4 import BeautifulSoup
    import re
    """
    Given a public Google Drive folder ID, fetch the embedded folder view
    and create a dictionary mapping each file name to its file ID.
    """
    # URL for the embedded folder view
    folder_url = f"https://drive.google.com/embeddedfolderview?id={folder_id}"
    response = requests.get(folder_url)
    
    if response.status_code != 200:
        raise Exception("Failed to fetch the folder page. Make sure the folder is public.")
    
    soup = BeautifulSoup(response.content, 'html.parser')
    files_dict = {}
    
    # Iterate through all links; each should represent a file in the folder
    for link in soup.find_all('a'):
        file_name = link.text.strip()
        href = link.get('href')
        if not href:
            continue
        
        # The href should include the file id in the format: /file/d/FILE_ID/view
        match = re.search(r'/file/d/([a-zA-Z0-9_-]+)/', href)
        if match:
            file_id = match.group(1)
            files_dict[file_name] = file_id

    return files_dict

## Algoritmo de detección de ruido


In [4]:
def noise_detection_algorithm(X, y, num_veces, con, metodo, logger): 
        
    from sklearn.model_selection import StratifiedKFold
    from tqdm import tqdm
    import numpy as np
    
    """
    Método para clasificar errores basado en el K-means:
    num_veces : numero de veces que se aplica el algoritmo
    pr : 0 en caso de que el dataframe no tenga ruida/ <0 para meter ruido en el dataframe con esa proporción
    con : numero entre [0,1] que mide del num_veces cuantos los tienen que detectar como error para que se reporte como ruido
    """
    n = len(y)
    n_mal = np.array(n*[0])
    n_splits = 5 #utilizamos para entrenar un 80% de los datos diviendolo en 7 conjuntos o grupos de datos

    for i in range(num_veces):
        logger.debug(f"Iteración {i}")
        particiones = StratifiedKFold(n_splits=n_splits, shuffle=True)     
        for j, (train_index, test_index) in enumerate(particiones.split(X, y)):        
            X_train = X[train_index,]
            y_train = y[train_index]
            X_test= X[test_index,]
            y_test = y[test_index]
            
            # generar modelo y predecir
            modelo = metodo.fit(X_train,y_train)
            y_pred = modelo.predict(X_test)
        
            # apuntamos cuáles han sido mal clasificados
            filtro = y_pred != y_test
            i_mal = test_index[filtro]
            n_mal[i_mal] = n_mal[i_mal]+1
            
    errores = n_mal > num_veces*con
    return errores


## T student

In [2]:
def t_student(data,confidence_level):

    """
    Funcion que dados un vector con valores, te calcula el intervalo de confianza mediante la T de student
    data : vector con los valores
    confidence_level : nivel de confianza del intervalo
    """

    import numpy as np
    from scipy.stats import t
    
    # Calculate sample mean and standard deviation
    mean = np.mean(data)
    std_dev = np.std(data, ddof=1)  # Use ddof=1 for sample standard deviation
    n = len(data)  # Sample size
    
    alpha = 1 - confidence_level
    
    # Degrees of freedom
    df = n - 1
    
    # t critical value for two-tailed test
    t_critical = t.ppf(1 - alpha/2, df)
    
    # Margin of error
    margin_of_error = t_critical * (std_dev / np.sqrt(n))
    
    # Confidence interval
    confidence_interval = (mean - margin_of_error, mean + margin_of_error)
    return confidence_interval

In [3]:
t_student([
         0.8506387160170324,
         0.9623140495867769,
         0.8260427263479145,
         0.8687068114511353,
         0.8319286068418443,
         0.7819572840293274,
         0.8492561983471074,
         0.8660624370594159,
         0.9437314906219151,
         0.815900944933203,
         0.8199052132701422,
         0.9626596790042581,
         0.7692307692307692,
         0.9431704885343968,
         0.9428812131423757,
         0.8869421487603306,
         0.8506387160170324,
         0.8673978065802592,
         0.8673978065802592,
         0.886317907444668,
         0.8869421487603306,
         0.9070753179002282,
         0.797676669893514,
         0.8869047619047619,
         0.9621059691482227
      ],0.95)

(0.8499588253632513, 0.8967440451493263)

## Calcula el kappa sobre el dataframe entero

In [6]:
def kappa_dataset_completo(X, y, k, model):

    from sklearn.metrics import cohen_kappa_score
    from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
    import numpy as np
    from sklearn.model_selection import StratifiedKFold
    from matplotlib import pyplot as plt
    from sklearn.metrics import classification_report
    from sklearn.model_selection import train_test_split

    skf = StratifiedKFold(n_splits=k, shuffle=True)

    # Inicializar matriz de confusión acumulada y lista de kappa
    # conf_matrix_total = np.zeros((len(np.unique(y)), len(np.unique(y))))
    kappa_scores = []

    # Loop sobre los folds
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index,], X[test_index,]
        y_train, y_test = y[train_index], y[test_index]

        # Entrenar modelo
        model.fit(X_train, y_train)

        # Predicción
        y_pred = model.predict(X_test)

        # Calcular matriz de confusión y kappa
        # conf_matrix = confusion_matrix(y_test, y_pred)
        # conf_matrix_total += conf_matrix
        kappa_scores.append(cohen_kappa_score(y_test, y_pred))

    # Kappa promedio
    kappa_mean = np.mean(kappa_scores)

    # disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_total/k ,
    #                               display_labels=model.classes_)
    # disp.plot()

    # plt.show()

    return kappa_mean

## Datasets y algoritmos de aprendizaje automático


In [ ]:
global datasets, metodos
from ipynb.fs.full.datasets import *
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
#Diccionario con los datasets de los experimentos
datasets = {"mnist_numbers" : mnist_numbers(), "mnist_with_clothes" : mnist_with_clothes(), "chinese_mnist" : chinese_mnist(),   \
            "wine" : wine(), "gamma_telescope" : gamma_telescope(),   \
            "image_segmentation": image_segmentation(), "digits_dataset": digits_dataset(), "breast_cancer": load_breast_cancer()}

# Algoritmos de clasificación utilizados durante los experimentos de la libreria sklearn
metodos = {"SGDClassifier" : SGDClassifier(), "gaussianNB": GaussianNB(), "random_forest": RandomForestClassifier(),  \
           "logistic_regression": LogisticRegression(),"xgboost": GradientBoostingClassifier(), "mlp_classifier" : MLPClassifier()}

# Código donde se realiza cada experimento




In [ ]:
def experimento(metodo_exp, parametros, df_str, now, logger):

    import numpy as np
    import os
    import csv
    from datetime import datetime
    import time 
    import json
    
    """
    Función que realiza un experimento estructurado de la siguiente manera para cada dataset->
    1. Introduce ruido en las etiquetas del dataset
    2. Aplica a todo el dataset el algoritmo de detección de ruido y obtiene una matriz de confusion
    3. Aplica el algoritmo de detección de ruido y obtiene intervalos de confianza para los kappas
    
    Parametros de entrada de la función->
    metodo_exp : metodo utilizado en el experimento
    parametros : hiperparametros elegidos para el experimento
    datasets_exp : lista con el nombre de los dataframes que se utilizaran en el experimento
    now : fecha y hora del comienzo de experimento para utilizarlo de nombre del archivo donde se guardaran los resultados
    """
    
    start_time = time.time()

    #Parámetros
    metodo = metodos.get(metodo_exp)
    ruido = parametros[0]
    n_val_cruz = int(parametros[1])
    cond = parametros[2]
    confidence_level = parametros[3]
    list_kappa_pp = []
    list_kappa_pd = []
    list_kappa_dp = []
    list_kappa_dd = []

    date_time = now.strftime("%m/%d/%Y")
    file_name = now.strftime("_%m_%d_%Y_%H_%M_%S")
    path = df_str + "_" + f"{ruido}" + "_" + metodo_exp + "_" + file_name + ".json"
    path2 = "ruido" + file_name + ".csv"
    # open(path, "w") #Esto se puede poner mas adelante
    dic = {}

    logger.info("Dataset : " + df_str)

    df = datasets.get(df_str)

    dic["metodo"] = metodo_exp
    dic["dataset"] = df_str
    dic["parametros"] = parametros
    
    #Metemos ruido en el dataset y aplicamos el algoritmo de detección de ruido
    
    # mnist = experimento_con_ruido(df,"label",metodo,path,df_str)
    mnist = experimento_con_ruido(df,"label",metodo,df_str)

    logger.info(f"Leemos el fichero con ruido de la carpeta de google drive para una proporcion de {ruido} de ruido y el dataset: " + df_str)
    mnist.meter_ruido(ruido)
    
    #Aplicamos el algoritmo a todos los datos y obtenemos la matriz de confusión
    logger.info("Kappas para dataset completo con y sin ruido")
    # mnist.errores = noise_detection_algorithm(mnist.X, mnist.y, n_val_cruz, cond, metodo,logger)
    kappa_sin, kappa_con = mnist.est(logger)

    confidence_interval_sin = t_student(kappa_sin,confidence_level)
    confidence_interval_con = t_student(kappa_con,confidence_level)
    
    #Calculamos los kappas de PvsP, DvsP, PvsD y DvsD para cada dataset y los metemos en una lista
    logger.info("Estadísticas del algoritmo de detección de ruido")
    kappa_pp,kappa_pd,kappa_dp,kappa_dd = mnist.estadisticas_noise_algorithm(n_val_cruz, cond, metodo,logger)
    # list_kappa_pp = list_kappa_pp + kappa_pp
    # list_kappa_pd = list_kappa_pd + kappa_pd
    # list_kappa_dp = list_kappa_dp + kappa_dp
    # list_kappa_dd = list_kappa_dd + kappa_dd

    #Calculamos los intervalos de confianza para cada lista de kappas
    #PvsP
    confidence_interval_pp = t_student(kappa_pp,confidence_level)
    
    #PvsD
    confidence_interval_pd = t_student(kappa_pd,confidence_level)
    
    #DvsP
    confidence_interval_dp = t_student(kappa_dp,confidence_level)
    
    #DvsD
    confidence_interval_dd = t_student(kappa_dd,confidence_level)

    end_time = time.time()

    # Calculate elapsed time
    elapsed_time = end_time - start_time
    
    # Convert to hours, minutes, and seconds
    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = int(elapsed_time % 60)

    tiempo = f"{hours}h_{minutes}m_{seconds}s"

    dic["tiempo"] = tiempo

    dic["sin_ruido"] = {"kappas":kappa_sin,"intervalo": confidence_interval_sin}
    dic["con_ruido"] = {"kappas":kappa_con,"intervalo": confidence_interval_con}

    print("Kappa de Cohen promedio sin ruido:", confidence_interval_sin)
    print("Kappa de Cohen promedio con ruido:", confidence_interval_con)

    dic["PvsP"] =  {"kappas": kappa_pp,"intervalo" :confidence_interval_pp}
    dic["PvsD"] =  {"kappas": kappa_pd,"intervalo" :confidence_interval_pd}
    dic["DvsP"] =  {"kappas": kappa_dp,"intervalo" :confidence_interval_dp}
    dic["DvsD"] =  {"kappas": kappa_dd,"intervalo" :confidence_interval_dd}

    json_object = json.dumps(dic, indent=3)    
 
    # Writing to sample.json
    with open(path, "w") as outfile:
        outfile.write(json_object)
    
    return path, date_time, path2

## Función para hacer los experimentos en periodos de desconexión

In [17]:
import pandas as pd
from datetime import datetime
from ipynb.fs.full.datasets import * 
import logging

def experimentos_no_realizados():
    
    """
    Función que parte de una base de experimentos en un excel llamado "experimentos.xlsx", y va realizando los experimentos no empezados
    de forma secuencial
    """
   
    experimentos = pd.read_excel("experimentos.xlsx")
    experimentos_no_hechos = experimentos[experimentos["FECHA"].isna()]
    # print(experimentos_no_hechos.head())

    # Create a logger
    logger = logging.getLogger("my_logger")
    logger.setLevel(logging.DEBUG)  # Set the logging level
    
    # Create a FileHandler to write logs to a file
    file_handler = logging.FileHandler("experimentos.log")
    file_handler.setLevel(logging.DEBUG)  # Set handler logging level
    
    # Create a log message format
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)  # Attach formatter to handler
    
    # Add handler to the logger
    logger.addHandler(file_handler)
    
    for index, row in experimentos_no_hechos.iterrows():

        #Datasets elegidos para el experimento
        if experimentos.at[index, "DATASETS"] == "all":
            # Se genera una lista con los nombres de todos los datasets
            datasets_exp = list(datasets.keys())
        else:
            # Obtenemos una lista con los nombres de los datasets seleccionados para ese experimento
            datasets_exp = experimentos_no_hechos.at[index,"DATASETS"].split(",")

        # Parametros que utilizaremos para el experimento
        parametros = [float(num) for num in str(experimentos_no_hechos.at[index,"PARAMETROS"]).split(",")]
        # print(experimentos_no_hechos.at[index,"METODO"])

        #Metodo que utilizaremos durante el experimento
        metodo = experimentos_no_hechos.at[index,"METODO"]

        #Empezamos el experimento apuntando en el archivo ".log" lo que vamos a realizar
        exp_str = f"Empezamos el experimento con el método " + metodo + " y parametros " + experimentos_no_hechos.at[index,"PARAMETROS"]
        logger.critical(exp_str)

        #Tiempo actual
        now = datetime.now()
        
        # Realizamos el experimento
        file_name, date_time, file_con_ruido = experimento(metodo,parametros,datasets_exp[0],now,logger)

        # Actualizamos el dataframe con el registro de los experimentos
        experimentos.loc[index, "FECHA"] = date_time  
        experimentos.loc[index, "NOMBRE ARCHIVO"] = file_name
        experimentos.loc[index, "NOMBRE ARCHIVO RUIDO"] = file_con_ruido
        experimentos.loc[index, "ITERACIONES"] =  int(parametros[1])*5

        #Una vez finalizado el experimento, actualizamos el excel con los resultados obtenidos
        experimentos.to_excel("experimentos.xlsx", index=False)
    file_handler.close()
    logger.removeHandler(file_handler)
    print("Terminados todos los experimentos")
    return 1

In [18]:
experimentos_no_realizados()

Kappa de Cohen promedio sin ruido: (0.9250831916612469, 0.9432725736509661)
Kappa de Cohen promedio con ruido: (0.4707757068726755, 0.5397219424637951)


C:\Users\Galo\AppData\Local\Temp\ipykernel_33136\3671143120.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '03/10/2025' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  experimentos.loc[index, "FECHA"] = date_time
C:\Users\Galo\AppData\Local\Temp\ipykernel_33136\3671143120.py:61: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'breast_cancer_0.15_SGDClassifier__03_10_2025_16_37_57.json' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  experimentos.loc[index, "NOMBRE ARCHIVO"] = file_name
C:\Users\Galo\AppData\Local\Temp\ipykernel_33136\3671143120.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'ruido_03_10_2025_16_37_57.csv' has dtype incompatible with 

Kappa de Cohen promedio sin ruido: (0.9136288964829815, 0.9198965702380564)
Kappa de Cohen promedio con ruido: (0.5997890545410725, 0.6280273673080206)
Kappa de Cohen promedio sin ruido: (0.9272434414757298, 0.934548698637675)
Kappa de Cohen promedio con ruido: (0.6268988772870326, 0.6398183467988653)
Kappa de Cohen promedio sin ruido: (0.8532668485898559, 0.8602153737229691)
Kappa de Cohen promedio con ruido: (0.5741317069061382, 0.580506160355833)
Kappa de Cohen promedio sin ruido: (0.9344047151758943, 0.9437474683399139)
Kappa de Cohen promedio con ruido: (0.6385434249228368, 0.6471904907305945)
Terminados todos los experimentos


1

## Funcion para cerrar los handlers cuando surga un error en el código (no forma parte del programa si no hay fallos)

In [ ]:
# import logging

# def close_all_handlers():
#     """Closes and removes all handlers from all loggers."""
#     for logger_name in logging.root.manager.loggerDict:
#         logger = logging.getLogger(logger_name)
#         if hasattr(logger, "handlers"):  # Check if the logger has handlers
#             for handler in logger.handlers[:]:  # Iterate over a copy of the handlers list
#                 handler.close()
#                 logger.removeHandler(handler)

# # Example usage
# close_all_handlers()

In [ ]:
    # #File donde se almacenara los resultados
    # with open(path, mode='a', newline='') as file:
    #     writer = csv.writer(file)
    #     writer.writerow([df_str])

    # with open(path, mode='a', newline='') as file:
    #     writer = csv.writer(file)
    #     writer.writerow(["Confidence interval of PvsP"])
    #     writer.writerow(confidence_interval_pp)
    #     writer.writerow(["Confidence interval of PvsD"])
    #     writer.writerow(confidence_interval_pd)
    #     writer.writerow(["Confidence interval of DvsP"])
    #     writer.writerow(confidence_interval_dp)
    #     writer.writerow(["Confidence interval of DvsD"])
    #     writer.writerow(confidence_interval_dd)
    
    # #Calculamos los intervalos de confianza para cada lista de kappas
    # #PvsP
    # confidence_interval_pp = t_student(list_kappa_pp,confidence_level)
    
    # #PvsD
    # confidence_interval_pd = t_student(list_kappa_pd,confidence_level)
    
    # #DvsP
    # confidence_interval_dp = t_student(list_kappa_dp,confidence_level)
    
    # #DvsD
    # confidence_interval_dd = t_student(list_kappa_dd,confidence_level)
    
    #Escribimos los resultados en el excel
    # with open(path, mode='a', newline='') as file:
    #     writer = csv.writer(file)
    #     writer.writerow(["Intervalos de confianza de todos los kappas y datasets"])
    #     writer.writerow(["Confidence interval of PvsP"])
    #     writer.writerow(confidence_interval_pp)
    #     writer.writerow(["Confidence interval of PvsD"])
    #     writer.writerow(confidence_interval_pd)
    #     writer.writerow(["Confidence interval of DvsP"])
    #     writer.writerow(confidence_interval_dp)
    #     writer.writerow(["Confidence interval of DvsD"])
    #     writer.writerow(confidence_interval_dd)